In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

# 加载数据
cloud_data = np.load("../data/processed/avg_cloud_thickness.npy")
slope_data = np.load("../data/processed/slope_resampled.npy")

# 确保数据形状一致
min_shape = (min(cloud_data.shape[0], slope_data.shape[0]),
             min(cloud_data.shape[1], slope_data.shape[1]))
cloud_data = cloud_data[:min_shape[0], :min_shape[1]]
slope_data = slope_data[:min_shape[0], :min_shape[1]]

# 设定标签
labels = (cloud_data < 10).astype(int)

# 组织数据格式
X = np.stack([cloud_data, slope_data], axis=0)  # (2, H, W)
X = torch.tensor(X, dtype=torch.float32).unsqueeze(0)  # 增加 batch 维度 (1, 2, H, W)
Y = torch.tensor(labels, dtype=torch.float32).unsqueeze(0)  # 保持空间维度 (1, H, W)

# 创建数据加载器
dataset = TensorDataset(X, Y)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# 定义全卷积网络模型
class SolarNet(nn.Module):
    def __init__(self):
        super(SolarNet, self).__init__()
        # 编码器
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        # 解码器
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.up2 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.up3 = nn.ConvTranspose2d(32, 1, kernel_size=2, stride=2)
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # 编码器
        x = self.pool(torch.relu(self.conv1(x)))  # (32, H/2, W/2)
        x = self.pool(torch.relu(self.conv2(x)))  # (64, H/4, W/4)
        x = self.pool(torch.relu(self.conv3(x)))  # (128, H/8, W/8)
        
        # 解码器
        x = torch.relu(self.up1(x))  # (64, H/4, W/4)
        x = torch.relu(self.up2(x))  # (32, H/2, W/2)
        x = self.up3(x)              # (1, H, W)
        return self.sigmoid(x)

# 初始化模型、损失函数和优化器
model = SolarNet()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
epochs = 10
for epoch in range(epochs):
    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        # 调整标签维度以匹配输出形状 (1, 1, H, W)
        batch_y = batch_y.unsqueeze(1)
        # 将标签调整为与输出相同的尺寸
        batch_y = nn.functional.interpolate(batch_y, size=outputs.shape[2:], mode='nearest')
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# 保存模型
torch.save(model.state_dict(), "../result/solar_model.pth")

Epoch 1/10, Loss: 0.6831
Epoch 2/10, Loss: 0.6304
Epoch 3/10, Loss: 0.5499
Epoch 4/10, Loss: 0.4381
Epoch 5/10, Loss: 0.3147
Epoch 6/10, Loss: 0.2030
Epoch 7/10, Loss: 0.1083
Epoch 8/10, Loss: 0.0404
Epoch 9/10, Loss: 0.0088
Epoch 10/10, Loss: 0.0012
